# Zonal Stats over time

Inputs:
* Product: `rainfall_grids_1998_2017`
* Variable: `rainfall`
* Aggregate Function: `mean`
* Zones: `KHM_Catch8_m_del.shp` or `KHM_Basin_Simple_A.shp` 

In [1]:
%matplotlib inline
import fiona
import rasterio.features
import xarray as xr
import rasterio.features
import xarray
import datacube
dc = datacube.Datacube(config='/g/data/u46/users/ext547/ewater/cambodia_cube/cambodia.conf')


from shapely.geometry import asShape
from shapely.geometry import MultiPolygon, Polygon

import pandas as pd
import dask
from distributed import Client, LocalCluster

# Specify location and name of catchment shapefile

In [2]:
shape_file = '/g/data/u46/users/ext547/ewater/input_data/Cambodia_boundary/KHM_Basin_Simple_A.shp'
#shape_file = '/g/data/u46/users/ext547/ewater/input_data/laos/boundary/ss/All_basins_in_laos2016/All_basins_in_laos2016_4326.shp'

# shape_file = '/g/data/u46/users/ext547/ewater/input_data/laos/boundary/Lao_62basin_plus_Mekong_4326.shp'

# define functions

In [9]:
def geometry_mask(geoms, geobox, all_touched=False, invert=False):
    """
    Create a mask from shapes.

    By default, mask is intended for use as a
    numpy mask, where pixels that overlap shapes are False.
    :param list[Geometry] geoms: geometries to be rasterized
    :param datacube.utils.GeoBox geobox:
    :param bool all_touched: If True, all pixels touched by geometries will be burned in. If
                             false, only pixels whose center is within the polygon or that
                             are selected by Bresenham's line algorithm will be burned in.
    :param bool invert: If True, mask will be True for pixels that overlap shapes.
    """
    data = rasterio.features.geometry_mask([geom.to_crs(geobox.crs) for geom in geoms],
                                           out_shape=geobox.shape,
                                           transform=geobox.affine,
                                           all_touched=all_touched,
                                           invert=invert)
    coords = [xr.DataArray(data=coord.values, name=dim, dims=[dim], attrs={'units': coord.units}) 
              for dim, coord in geobox.coords.items()]
    return xarray.DataArray(data, coords=coords)

In [10]:
def get_shapes(shape_file):
    with fiona.open(shape_file) as shapes:
        crs = datacube.utils.geometry.CRS(shapes.crs_wkt)
        for shape in shapes:
            geom = datacube.utils.geometry.Geometry(shape['geometry'], crs=crs)
            yield geom, shape['properties']

# Set up dask

In [12]:
# cluster = LocalCluster(local_dir='/local/u46/adh547/tmp')
cluster = LocalCluster(local_dir='/g/data/u46/users/ext547/ewater/working')

client = Client(cluster)
dask.config.set(get=client.get)
client

Client Scheduler: tcp://127.0.0.1:42622 Dashboard: http://127.0.0.1:39507/status,Cluster Workers: 8 Cores: 8 Memory: 33.67 GB


#  Loop through catchments

## Set up catchment data

In [13]:
dc.list_products()
# dc.list_measurements()

,name,description,instrument,creation_time,product_type,format,lat,label,platform,time,lon,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,
17,alos_fnf_mosaic,PALSAR-2/PALSAR/JERS-1 Mosaic and Forest/Non-F...,"PALSAR, PALSAR-2",None,SAR_fnf,NETCDF,None,None,ALOS,None,None,EPSG:4326,"(-0.000222222222222, 0.000222222222222)",None,"(latitude, longitude)"
16,alos_sar_mosaic,PALSAR-2/PALSAR/JERS-1 Mosaic and Forest/Non-F...,"PALSAR, PALSAR-2",None,SAR,NETCDF,None,None,ALOS,None,None,EPSG:4326,"(-0.000222222222222, 0.000222222222222)",None,"(latitude, longitude)"
10,dem_3sec_hydrosheds,three sec hydrologically conditioned Digital E...,shuttle radar,None,elevation,NETCDF,None,None,elevation,None,None,EPSG:4326,"(-0.00083, 0.00083)",None,"(latitude, longitude)"
6,ls5_usgs_sr_scene,Landsat 5 USGS Collection 1 Level2 Surface Ref...,TM,None,LEDAPS,GeoTiff,None,None,LANDSAT_5,None,None,"PROJCS[""WGS 84"",GEOGCS[""WGS 84"",DATUM[""WGS_198...","(-30, 30)",None,"(y, x)"
2,ls7_usgs_sr_scene,Landsat 7 USGS Collection 1 Level2 Surface Ref...,ETM,None,LEDAPS,GeoTiff,None,None,LANDSAT_7,None,None,"PROJCS[""WGS 84"",GEOGCS[""WGS 84"",DATUM[""WGS_198...","(-30, 30)",None,"(y, x)"
1,ls8_usgs_sr_scene,Landsat 8 USGS Collection 1 Level2 Surface Ref...,OLI_TIRS,None,LaSRC,GeoTiff,None,None,LANDSAT_8,None,None,"PROJCS[""WGS 84"",GEOGCS[""WGS 84"",DATUM[""WGS_198...","(-30, 30)",None,"(y, x)"
5,ls_level2_geomedian_annual,Landsat Level-2 Annual GeoMedian,"TM,ETM,OLI_TIRS",None,surface_reflectance_statistical_summary,GeoTiff,None,None,"LANDSAT_5,LANDSAT_7,LANDSAT_8",None,None,EPSG:32648,"(-30, 30)","(30720.0, 30720.0)","(y, x)"
9,pet_grids_1979_2016,Daily potential evapo-transpiration (PET) for ...,modelled,None,PET,NETCDF,None,None,PET,None,None,EPSG:4326,"(-0.5, 0.5)",None,"(latitude, longitude)"
18,rainfall_grids,bias corrected rainfall data for Cambodia prov...,rain gauge,None,rainfall,NETCDF,None,None,rain,None,None,EPSG:4326,"(-0.25, 0.25)",None,"(latitude, longitude)"


In [26]:
product_id = 'spei_grids'
measurement_id = 'spei_gamma_03'

In [27]:
product = dc.index.products.get_by_name(product_id)
datasets = dc.find_datasets(product=product_id)
crs = product.grid_spec.crs
resolution = product.grid_spec.resolution
align = product.grid_spec.alignment

crs, resolution, align

(CRS('EPSG:4326'), (-0.5, 0.5), (0.0, 0.0))

In [37]:
upsample = 0.01 #rainfall
# upsample = 1 #SPEI
hi_resolution = [r * upsample for r in resolution]
hi_resolution

[-0.005, 0.005]

In [38]:
# shapes = get_shapes(shape_file)

## load data for catchments

In [43]:
shapes = get_shapes(shape_file)
loaded_xr = {}
for geometry, properties in shapes:
    geobox = datacube.utils.geometry.GeoBox.from_geopolygon(geometry, hi_resolution, crs, align)
    data = None
    data = dc.load(product=product_id, 
               measurement=measurement_id, 
               datasets=datasets, 
               dask_chunks={'time': 1}, 
               geopolygon=geometry,
               resolution=hi_resolution,)
               #output_crs='EPSG:32648')
    SCID = properties['SCID']
    print(SCID)
    mask = geometry_mask([geometry], geobox, all_touched=True, invert=True)
    rain_array = data.spei_pearson_03.where(data.spei_pearson_03 > -3.1).where(mask)
#     rain_array = data.spei_gamma_03.where(data.spei_gamma_03 > -3.1).where(mask)
    loaded = None
    loaded = rain_array.mean(dim=['latitude', 'longitude']).load();
    del data
    loaded_xr[SCID] = loaded
    col = loaded.time.values
    del loaded
    
# print(loaded_xr)

# loaded_pd = pd.DataFrame.from_dict(loaded_xr)

# col = loaded.time.values
# loaded_pd.index = col.astype('datetime64[D]')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [44]:
loaded_pd = pd.DataFrame.from_dict(loaded_xr)

# col = loaded.time.values
loaded_pd.index = col.astype('datetime64[D]')

In [45]:
loaded_pd

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
2000-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-31,-0.671256,0.013750,-0.076525,-0.435386,-1.096649,0.191523,-0.295234,-1.169837,-0.557585,-1.199729,...,-0.741765,-0.917931,-0.696157,-0.661064,-1.107220,-0.383390,-0.543918,-0.458475,-1.010154,-1.068956
2000-04-30,0.386576,-0.390282,-0.332968,-0.172088,0.440533,0.025746,-0.282122,-0.661474,-0.362295,-0.647705,...,0.115186,-0.476455,0.000305,0.095077,-0.733763,-0.597575,0.579289,-1.107193,0.253394,-1.214777
2000-05-31,0.050206,-1.114367,-0.716342,-0.186732,0.250662,0.074939,-0.521659,-0.624009,-0.253565,-0.426038,...,-0.586363,-1.077197,-0.120946,-0.563130,-0.658907,-0.809820,0.611670,-1.070064,0.440336,-1.353966
2000-06-30,0.532280,-1.527690,-1.018828,0.233183,0.412718,0.341555,-0.275086,-0.260045,0.007043,0.016906,...,-0.457166,-0.887170,0.326976,-0.415831,-0.770681,-0.746797,0.325750,-0.935699,0.649790,-1.560674
2000-07-31,-0.191303,-1.327170,-1.059213,-0.300304,-0.492050,0.072686,-0.336234,-0.138649,-0.040717,-0.285206,...,-1.247709,-0.990272,-0.486291,-1.006661,-1.043064,-1.077709,-0.263039,-0.950826,-0.749687,-1.536037
2000-08-31,-1.581959,-1.386767,-1.330849,-0.710980,-1.107923,-0.326310,-0.372081,-0.402371,-0.898605,-0.326462,...,-1.349216,-1.075414,-0.806928,-1.202166,-1.460318,-1.330349,-1.520782,-1.313993,-1.414729,-1.150006
2000-09-30,-1.572252,-1.306031,-1.598228,-1.550120,-1.432582,-1.195379,-0.746156,-0.942889,-1.249149,-0.976288,...,-1.446439,-1.379057,-1.240411,-1.517521,-1.855771,-1.621406,-1.147187,-1.423601,-1.741504,-1.153845
2000-10-31,-1.295506,-0.706558,-1.172161,-0.750729,-1.484732,-0.238484,-0.100952,-0.460017,-0.777340,-0.110455,...,-0.451055,-0.005277,-0.599664,-0.755287,-0.760931,-1.117980,-0.939102,-1.134233,-1.227399,-0.528229


In [46]:
csv_out = '/g/data/u46/users/ext547/ewater/output_data/catchment_output/'+str(product_id)+'_pearson_outfile.csv'
loaded_pd.to_csv(csv_out)

In [ ]:
# loaded_pd = pd.DataFrame.from_dict(loaded_xr)#, orient='index')
# loaded_pd.index = col.astype('datetime64[D]')

# masked_pd = loaded_pd.where(loaded_pd != 0.000000)

In [ ]:
# loaded_xr = {}
# for geometry, properties in shapes:
#     geobox = datacube.utils.geometry.GeoBox.from_geopolygon(geometry, hi_resolution, crs, align)
#     sensor_nbar = dc.load(product=product_id, 
#                measurement=measurement_id, 
#                datasets=datasets, 
#                dask_chunks={'time': 1}, 
#                geopolygon=geometry,
#                group_by='solar_day', 
#                resolution=hi_resolution,)
#     #retrieve the projection information before masking/sorting
#     crs = sensor_nbar.crs
#     crswkt = sensor_nbar.crs.wkt
#     affine = sensor_nbar.affine
#     #assign pq data variable
#     sensor_pq= sensor_nbar.pixel_qa
#     mask_components = {'cloud_shadow': 'no_cloud_shadow',
#                            'cloud': 'no_cloud',}
#     good_data = masking.make_mask(sensor_pq, **mask_components)
# #     good_data = quality_mask.loc[start_of_epoch:end_of_epoch]
#     sensor_nbar2 = sensor_nbar.where(good_data)
#     del sensor_nbar
#     del good_data
#     SCID = properties['SCID']    

#     print(SCID)
    
#     mask = geometry_mask([geometry], geobox, all_touched=True, invert=True)
#     data_array = None
#     data_array = (sensor_nbar2.nir - sensor_nbar2.red)/(sensor_nbar2.nir+sensor_nbar2.red)
#     data_array = data_array.where(mask)
#     loaded_array = None
#     loaded_array = data_array.mean(dim=['y', 'x'])
#     col = loaded_array.time.values
# #     print(col)
#     loaded_array02 = None
#     loaded_array02 = loaded_array.load()
#     loaded_array02['ID'] = ('time', np.repeat(SCID, loaded_array02.time.size))
    
#     del data_array
    
#     loaded_xr[SCID] = loaded_array02

    
#     del loaded_array
#     del loaded_array02

# # loaded_clean = xr.concat(loaded_xr.values(), 'time')
# # loaded_clean = loaded_clean.sortby('time')
# # loaded_clean.attrs['crs'] = crs
# # loaded_clean.attrs['affin|e'] = affine   
    
# # 
# #     loaded_pd = pd.DataFrame.from_dict(loaded_xr)
# #     loaded_pd.index = col.astype('datetime64[D]')

In [ ]:
# loaded_pd.min()

In [ ]:
loaded_xr

In [ ]:
# loaded_pd = pd.DataFrame.from_dict(loaded_xr)
# loaded_pd.head()

In [ ]:
# ls7_masked_pd = loaded_pd.where(loaded_pd != 0.000000)
# ls7_masked_pd

In [ ]:
ls5_masked_pd = loaded_pd.where(loaded_pd != 0.000000)
ls5_masked_pd.head()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
csv_out = '/g/data/u46/users/ext547/ewater/input_data/Cambodia_boundary/'+str(start_of_epoch)+'_'+str(product_id)+'_out.csv'
ls5_masked_pd.to_csv(csv_out)

In [ ]:
datacube.model.__file__

In [ ]:
for k, v in loaded_array.data.dask.items():
    v.__getstate__()
    

In [ ]:
stop

In [ ]:
loaded_pd

In [ ]:
csv_out = '/g/data/u46/users/ext547/ewater/input_data/Cambodia_boundary/'+str(measurement_id)+'_287_csv_out.csv'
loaded_pd.to_csv(csv_out)

In [ ]:
shapes = get_shapes(shape_file)

In [ ]:
geometry, properties = next(shapes)
print(f"{int(geometry.area / (1000*1000))} km^2")
asShape(geometry)

In [ ]:
geobox = datacube.utils.geometry.GeoBox.from_geopolygon(geometry, hi_resolution, crs, align)
mask = geometry_mask([geometry], geobox, all_touched=True, invert=True)

In [ ]:
properties

In [ ]:
mask

In [ ]:
mask.plot(size=6, aspect=(mask.shape[1]/mask.shape[0]), add_colorbar=False);

In [ ]:
asShape(geometry)

In [ ]:
data = dc.load(product=product_id, 
               measurement=measurement_id, 
               datasets=datasets, 
               dask_chunks={'time': 1}, 
               geopolygon=geometry,
               resolution=hi_resolution)

In [ ]:
data

In [ ]:
rain_array = data.spei_gamma_03 .where(data.spei_gamma_03  > -3.1).where(mask)

In [ ]:
rain_array

In [ ]:
rain_array.values.max()

In [ ]:
loaded = rain_array.mean(dim=['latitude', 'longitude']).load();

In [ ]:
loaded

In [ ]:
mean_rain = rain_array.mean(dim=['latitude', 'longitude'])
mean_rain

In [ ]:
mean_rain.values.max()

In [ ]:
mean_rain.isel(time=4).values

In [ ]:
mean_rain.isel(time=slice(10,15)).plot()

In [ ]:
rain_array[:20].load().plot(col='time', col_wrap=5, size=5, aspect=(mask.shape[1]/mask.shape[0]), add_colorbar=True)